<a href="https://colab.research.google.com/github/j-hay-214/osu-gradtda-5622-sp25/blob/main/_site/course_materials/hw/2/Jarrod_Hay_Homework2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GRADTDA5622 - Big Data Computing Foundations 2
## Homework 2: Manipulating Data with SQL
- Semester: Spring 2025
- Instructor: Tom Bihari
- Section: N/A
- Student Name: Jarrod Hay
- Student Email: jarrod.hay@osumc.edu | hay.177@osu.edu
- Student ID: 500287277
***

***
# Section: Overview
***

**The Objectives of This Assignment are:**
1. To gain experience using SQL to manipulate data to solve problems and answer questions.

**Instructions:**
- **Follow the instructions** in each section.
- **Fill in** the **Conclusions** section.

***
# Section: Setup
- Add any needed imports, helper functions, etc., here.
- **I added all of these helper functions... Tom Bihari**
***

In [1]:
import os
import pandas as pd
import sqlite3 as sq

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

data_directory = "../shared_Sp23/"  # Set this to the appropriate directory for your setup.

In [2]:
# Connect to a SQLite database (will be created if it doesn't exist).
def connect_db(file_name):
    if not os.path.isfile(file_name):
        print('Database file does not exist and will be created.')
    try:
        conn = sq.connect(file_name)
        print("Database connection established.")
        return conn
    except Error:
        print(Error)
        return None

In [3]:
# Create a cursor to use to access the data.
def get_cursor(connection):
    try:
        cur = connection.cursor()
        print('Cursor created.')
        return cur
    except Error:
        print(Error)
        return None

In [4]:
# Close the connection to the database.
def close_db(connection, cursor):
    connection.commit()
    cursor.close()
    connection.close()

In [5]:
# Create a SQLite table by reading a CSV file.  Uses Pandas.
def create_table_from_csv(connection, file_name, table_name):
    pdf = pd.read_csv(file_name)
    create_table_from_dataframe(connection, pdf, table_name)

In [6]:
# Create a SQLite table by reading a worksheet in an excel file.  Uses Pandas.
def create_table_from_excel(connection, file_name, sheet_name, table_name):
    pdf = pd.read_excel(file_name, sheet_name)
    create_table_from_dataframe(connection, pdf, table_name)

In [7]:
# Create a SQLite table from a Pandas DataFrame.
def create_table_from_dataframe(connection, pdf, table_name):
    pdf.to_sql(table_name, connection, if_exists='replace', index = False)

In [8]:
# Create a Pandas DataFrame from a SQLite cursor.
def create_dataframe_from_cursor(cursor):
    cols = [column[0] for column in cursor.description]
    return pd.DataFrame.from_records(data = cursor.fetchall(), columns = cols)

In [9]:
# Create a Pandas DataFrame from a SQLite table (or view).
#   Or could do: df = pd.read_sql_query("SELECT * FROM table_name", cnx)
#   See: https://stackoverflow.com/questions/36028759/how-to-open-and-convert-sqlite-database-to-pandas-dataframe
def create_dataframe_from_table(connection, table_name):
    cursor = run_query(connection,'SELECT * FROM "' + table_name + '"')
    cols = [column[0] for column in cursor.description]
    return pd.DataFrame.from_records(data = cursor.fetchall(), columns = cols)

In [10]:
# Run a SQLite query and return the result.
def run_query(conn_or_cur, query_string):
    try:
        result = conn_or_cur.execute(query_string)
    except:
        print("Query error")
        result = None
    return result

In [11]:
# Replace text \n and \t with actual newline and tab characters.
def clean_string(strng):
    return str(strng).replace('\\n', '\n').replace('\\t', '\t')

In [12]:
# Print query results up to a specified max number of records).
def print_result(cursor, max_num_records):
    print("------------------------------------------")
    df = create_dataframe_from_cursor(cursor)
    #print(df.head(max_num_records))
    display(df.head(max_num_records))
    print("------------------------------------------")

In [13]:
def print_all_table_and_view_names(conn_or_cur):
    result = run_query(conn_or_cur,'SELECT name from sqlite_master where type= "table"')
    print("\nALL TABLE NAMES: ",result.fetchall())
    result = run_query(conn_or_cur,'SELECT name from sqlite_master where type= "view"')
    print("ALL VIEW NAMES:  ",result.fetchall())

In [14]:
def print_schema(conn_or_cur, table_or_view_name):
    #https://www.sqlite.com/schematab.html
    # Table "sqlite_schema" is correct, but "sqlite_master" is used for backward compatibility.)
    result = run_query(conn_or_cur,'SELECT sql from sqlite_master where name= "' + table_or_view_name + '"')
    print("\nSCHEMA: " + table_or_view_name)
    print(clean_string(result.fetchall()))

In [15]:
def print_table_or_view(conn_or_cur, table_name, max_num_records):
    print("\nTABLE/VIEW: " + table_name)
    result = run_query(conn_or_cur,'SELECT * FROM "' + table_name + '"')
    print_result(result, max_num_records)

---
# Set up a Database
- Create a blank database.
- Create three tables from the **state-abbrevs.csv**, **state-areas.csv**, and **state-populations.csv** files in the shared data folder.
- **I have done these steps for you**, from the SQL Examples notebook.
---

## Connect to a blank database

In [16]:
# Connect to the database and get a cursor to access the data.
#conn = connect(r'SQLiteUSStatesData.db')  # a file database
conn = connect_db(r':memory:')             # an in-memory database
cur = get_cursor(conn)  # not needed for these examples, but kept for compatibility

Database file does not exist and will be created.
Database connection established.
Cursor created.


## Create some tables by reading data from CSV files

In [21]:
create_table_from_csv(conn, '/state-abbrevs.csv', 'abbrevs_table')
print_schema(conn,"abbrevs_table")
print_table_or_view(conn, "abbrevs_table", 3) # Show a few example records


SCHEMA: abbrevs_table
[('CREATE TABLE "abbrevs_table" (
"state" TEXT,
  "abbreviation" TEXT
)',)]

TABLE/VIEW: abbrevs_table
------------------------------------------


,state,abbreviation
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ


------------------------------------------


In [22]:
create_table_from_csv(conn, '/state-areas.csv', 'areas_table')
#print_schema(conn,"areas_table")
print_table_or_view(conn, "areas_table", 3) # Show a few example records


TABLE/VIEW: areas_table
------------------------------------------


,state,area
0,Alabama,52423
1,Alaska,656425
2,Arizona,114006


------------------------------------------


In [23]:
create_table_from_csv(conn, '/state-populations.csv', 'populations_table')
#print_schema(conn,"populations_table")
print_table_or_view(conn, "populations_table", 3) # Show a few example records


TABLE/VIEW: populations_table
------------------------------------------


,state,ages,year,population
0,AL,under18,2012,1117489.0
1,AL,total,2012,4817528.0
2,AL,under18,2010,1130966.0


------------------------------------------


In [ ]:
print_all_table_and_view_names(conn)

---
# Task 1: Create a Table of Summarized State Populations
- Get the min, average, and max total and under18 populations for each of the states over all years.
- Call the table **summarized_state_populations**.
- The table should have the following fields:
  - state, ages, min_pop, avg_pop, max_pop
  - It should be sorted by **state** (from A to Z), and within each state, by **under18** age range first, then **total** age range.
---

In [28]:
# Fill in the blanks
run_query(conn,'DROP TABLE IF EXISTS summarized_state_populations') # Delete the table if it already exists

run_query(conn,'''
    CREATE TABLE IF NOT EXISTS summarized_state_populations
    AS SELECT state, ages, min(population) AS min_pop, avg(population) AS avg_pop, max(population) AS max_pop
    FROM populations_table
    GROUP BY state, ages
    ORDER BY "state" ASC, "ages" DESC
    ''')

print_table_or_view(conn, "summarized_state_populations", 3)


TABLE/VIEW: summarized_state_populations
------------------------------------------


,state,ages,min_pop,avg_pop,max_pop
0,AK,under18,177502.0,1.866730e+05,192636.0
1,AK,total,553290.0,6.462048e+05,735132.0
2,AL,under18,1050041.0,1.111197e+06,1134927.0


------------------------------------------


---
# Task 2: Print the Top 3 States with the Largest Normalized Population Swings
- Consider the difference between max and min "total" populations for each state.
- Normalize by dividing by the average population for the state.
- You can use the table from the previous query.
---

In [31]:
# Fill in the blanks

result = run_query(conn,'''
    SELECT state, min_pop, max_pop, ((max_pop - min_pop)/avg_pop) AS norm_pop
    FROM summarized_state_populations
    WHERE ages = "total"
    ORDER BY "norm_pop" DESC
    ''')

print_result(result, 3)

------------------------------------------


,state,min_pop,max_pop,norm_pop
0,NV,1220695.0,2790136.0,0.748474
1,AZ,3684097.0,6626624.0,0.555760
2,UT,1731223.0,2900872.0,0.503931


------------------------------------------


---
# Task 3: Create a Table of Population Densities
- Follow each of the individual steps below.  There likely are shorter ways to do this task, but this way the steps are explicit.
- I have given the shells of some of the queries, so you can "fill in the blanks".
- I have included the expected outputs, so you can see if yours matches.
- Name the table **population_densities_table**
- The table should have the following fields:
  - state_abbrev
  - state_name
  - area
  - year
  - population_total
  - population_under_18
  - population_18_and_over
  - density_total
  - density_under_18
  - density_18_and_over
---

## Create a "pop_and_name_table"
- Join the populations_table and abbrevs_table.
- Table should have fields: state_abbrev, state_name, ages, year, population.

In [34]:
run_query(conn,'DROP TABLE IF EXISTS pop_and_name_table') # Delete the table if it already exists

result = run_query(conn,'''
    CREATE TABLE IF NOT EXISTS pop_and_name_table
    AS SELECT populations_table.state AS state_abbrev, abbrevs_table.state AS state_name, ages, year, population
    FROM (populations_table LEFT OUTER JOIN abbrevs_table ON populations_table.state = abbrevs_table.abbreviation)
    ''')

print_table_or_view(conn, "pop_and_name_table", 3) # Show a few records


TABLE/VIEW: pop_and_name_table
------------------------------------------


,state_abbrev,state_name,ages,year,population
0,AL,Alabama,under18,2012,1117489.0
1,AL,Alabama,total,2012,4817528.0
2,AL,Alabama,under18,2010,1130966.0


------------------------------------------


## Create a "pop_name_area_density_table"
- Join the pop_and_name_table and areas_table.
- Table should have fields: state_abbrev, state_name, ages, year, population, area, density.

In [36]:
run_query(conn,'DROP TABLE IF EXISTS pop_name_area_density_table') # Delete the table if it already exists

result = run_query(conn,'''
    CREATE TABLE IF NOT EXISTS pop_name_area_density_table
    AS SELECT pop_and_name_table.*, areas_table.area, (pop_and_name_table.population / areas_table.area) AS density
    FROM (pop_and_name_table LEFT OUTER JOIN areas_table ON pop_and_name_table.state_name = areas_table.state)
    ORDER BY year ASC, state_abbrev ASC, ages ASC
    ''')

print_table_or_view(conn, "pop_name_area_density_table", 3) # Show a few records


TABLE/VIEW: pop_name_area_density_table
------------------------------------------


,state_abbrev,state_name,ages,year,population,area,density
0,AK,Alaska,total,1990,553290.0,656425.0,0.842884
1,AK,Alaska,under18,1990,177502.0,656425.0,0.270407
2,AL,Alabama,total,1990,4050055.0,52423.0,77.257215


------------------------------------------


## Create an "under18_table"
- Select only the "under18" ages records from the pop_name_area_density_table.
- Table should have fields: state_abbrev, state_name, year, population, area, density.

In [39]:
run_query(conn,'DROP TABLE IF EXISTS under18_table') # Delete the table if it already exists

result = run_query(conn,'''
    CREATE TABLE IF NOT EXISTS under18_table
    AS SELECT state_abbrev, state_name, year, population, area, density
    FROM pop_name_area_density_table
    WHERE ages = "under18"
    ORDER BY year ASC, state_abbrev ASC
    ''')

print_table_or_view(conn, "under18_table", 3) # Show a few records


TABLE/VIEW: under18_table
------------------------------------------


,state_abbrev,state_name,year,population,area,density
0,AK,Alaska,1990,177502.0,656425.0,0.270407
1,AL,Alabama,1990,1050041.0,52423.0,20.030159
2,AR,Arkansas,1990,620933.0,53182.0,11.675623


------------------------------------------


## Create a "total_table"
- Select only the "total" ages records from the pop_name_area_density_table.
- Table should have fields: state_abbrev, state_name, year, population, area, density.

In [41]:
run_query(conn,'DROP TABLE IF EXISTS total_table') # Delete the table if it already exists

result = run_query(conn,'''
    CREATE TABLE IF NOT EXISTS total_table
    AS SELECT state_abbrev, state_name, year, population, area, density
    FROM pop_name_area_density_table
    WHERE ages = "total"
    ORDER BY year ASC, state_abbrev ASC
    ''')

print_table_or_view(conn, "total_table", 3) # Show a few records


TABLE/VIEW: total_table
------------------------------------------


,state_abbrev,state_name,year,population,area,density
0,AK,Alaska,1990,553290.0,656425.0,0.842884
1,AL,Alabama,1990,4050055.0,52423.0,77.257215
2,AR,Arkansas,1990,2356586.0,53182.0,44.311722


------------------------------------------


## Create a "total_and_under18_table"
- Join the under18_table and total_table.
- Table should have fields: state_abbrev, state_name, year, area, total_pop, total_density, under18_pop, under18_density.

In [42]:
run_query(conn,'DROP TABLE IF EXISTS total_and_under18_table') # Delete the table if it already exists

result = run_query(conn,'''
    CREATE TABLE IF NOT EXISTS total_and_under18_table
    AS SELECT total_table.state_abbrev, total_table.state_name, total_table.year, total_table.area,
        total_table.population AS total_pop, total_table.density AS total_density, under18_table.population AS under18_pop, under18_table.density AS under18_density
    FROM (total_table LEFT OUTER JOIN under18_table ON
        (total_table.state_abbrev = under18_table.state_abbrev AND total_table.year = under18_table.year))
    ''')

print_table_or_view(conn, "total_and_under18_table", 3) # Show a few records


TABLE/VIEW: total_and_under18_table
------------------------------------------


,state_abbrev,state_name,year,area,total_pop,total_density,under18_pop,under18_density
0,AK,Alaska,1990,656425.0,553290.0,0.842884,177502.0,0.270407
1,AL,Alabama,1990,52423.0,4050055.0,77.257215,1050041.0,20.030159
2,AR,Arkansas,1990,53182.0,2356586.0,44.311722,620933.0,11.675623


------------------------------------------


## Create a "total_under18_over18_table"
- Start with the total_and_under18_table and calculate columns for the over18_pop, over18_density.
- Actually, we mean "18 and over", not "over 18".
- Table should have fields: state_abbrev, state_name, year, area, total_pop, total_density, under18_pop, under18_density, over18_pop, over18_density.

In [43]:
run_query(conn,'DROP TABLE IF EXISTS total_under18_over18_table') # Delete the table if it already exists

result = run_query(conn,'''
    CREATE TABLE IF NOT EXISTS total_under18_over18_table
    AS SELECT *, (total_pop - under18_pop) AS over18_pop, (total_density - under18_density) as over18_density
    FROM total_and_under18_table
    ORDER BY year ASC, state_abbrev ASC
    ''')

print_table_or_view(conn, "total_under18_over18_table", 3) # Show a few records


TABLE/VIEW: total_under18_over18_table
------------------------------------------


,state_abbrev,state_name,year,area,total_pop,total_density,under18_pop,under18_density,over18_pop,over18_density
0,AK,Alaska,1990,656425.0,553290.0,0.842884,177502.0,0.270407,375788.0,0.572477
1,AL,Alabama,1990,52423.0,4050055.0,77.257215,1050041.0,20.030159,3000014.0,57.227057
2,AR,Arkansas,1990,53182.0,2356586.0,44.311722,620933.0,11.675623,1735653.0,32.636099


------------------------------------------


## Create the final population_densities_table"
- Start with the total_under18_over18_table.
- Rename and reorder the columns to match the original request.
- Sort by year and state abbreviation, ascending.
- Table should have fields: state_abbrev, state_name, area, year,
  - population_total, population_under_18, population_18_and_over, density_total, density_under_18, density_18_and_over.

In [44]:
run_query(conn,'DROP TABLE IF EXISTS population_densities_table') # Delete the table if it already exists

result = run_query(conn,'''
    CREATE TABLE IF NOT EXISTS population_densities_table
    AS SELECT state_abbrev, state_name, area, year, total_pop AS population_total, under18_pop AS population_under_18, over18_pop AS population_18_and_over,
        total_density AS density_total, under18_density as density_under_18, over18_density as density_18_and_over
    FROM total_under18_over18_table
    ORDER BY year ASC, state_abbrev ASC
    ''')

print_table_or_view(conn, "population_densities_table", 10) # Show a few records


TABLE/VIEW: population_densities_table
------------------------------------------


,state_abbrev,state_name,area,year,population_total,population_under_18,population_18_and_over,density_total,density_under_18,density_18_and_over
0,AK,Alaska,656425.0,1990,553290.0,177502.0,375788.0,0.842884,0.270407,0.572477
1,AL,Alabama,52423.0,1990,4050055.0,1050041.0,3000014.0,77.257215,20.030159,57.227057
2,AR,Arkansas,53182.0,1990,2356586.0,620933.0,1735653.0,44.311722,11.675623,32.636099
3,AZ,Arizona,114006.0,1990,3684097.0,1006040.0,2678057.0,32.314940,8.824448,23.490492
4,CA,California,163707.0,1990,29959515.0,7980501.0,21979014.0,183.006927,48.748685,134.258242
5,CO,Colorado,104100.0,1990,3307618.0,881640.0,2425978.0,31.773468,8.469164,23.304304
6,CT,Connecticut,5544.0,1990,3291967.0,752666.0,2539301.0,593.789141,135.762266,458.026876
7,DC,District of Columbia,68.0,1990,605321.0,112632.0,492689.0,8901.779412,1656.352941,7245.426471
8,DE,Delaware,1954.0,1990,669567.0,165628.0,503939.0,342.664790,84.763562,257.901228
9,FL,Florida,65758.0,1990,13033307.0,2988807.0,10044500.0,198.201086,45.451610,152.749475


------------------------------------------


---
# Cleanup and Close the Current Database
---

In [45]:
close_db(conn, cur)

---
# Task 4: Process Datasets with SQL and Answer Questions - From Scratch
In this Task, you will load several datasets into a new database.  You then will process the data and answer the following questions.  The code in Tasks 1-3 above can be used as examples.

The data is in an Excel file: 'MiscWikipediaCountryData.xls'.  You may open the file in Excel and look at it if you like.  The file contains four worksheets: GovernmentData, InternetData, IncomeEqualityData, and IncomePerCapitaData.

**Context and Hints**
- We are interested only in UN_Member countries.
- We don't care about the Year information.  We will assume that the data is relevant for the current year.
- Each dataset contains some of the information needed to answer the questions, so you will have to combine the information.  Consider building a single table "common_country_table" with columns:
  - **Country, Constitutional_Form, Head_Of_State, Subregion, Region, Population, Rich_Poor_Income_Ratio (use the CIA_RP10 column), Per_Capita_Income (use the IMF_Estimate column)**
  - Consider using JOIN operations.
- There is an issue with the data (as is usually the case...): Each dataset has a column that contains the country's name.  However, the datasets may contain different subsets of countries, and the country names may be spelled differently.  What should you do?  Here are some options:
  1. Treat one of the datasets (e.g., GovernmentData) as the primary dataset, and join the other datasets' data to the primary dataset IF you find a match; otherwise, join null information.
      - Pros: Gives a consistent set of countries.  Cons: Some fields will be null.
  2. Treat all of the datasets as "equal" and join information only if you find matching country names in all datasets.
      - Pros: No null fields, unless they were null in the original datasets.  Cons: A row will appear only if the country name appears in ALL datasets, so some countries will be missing!
  3. Do a deeper analysis and try to fix as many country name mismatches as possible.  Then join them.
      - Pros: May results in cleaner / more complete final product.  Cons: Difficult and probably manual work.
  4. Create a combined table of all of the country names across all of the datasets, and join all four datasets to that table.
      - Pros: Result will contain all information from all datasets.  Cons: A single real country may appear in multiple rows with different country names.

For simplicity, let's go with option 2.  What kind of JOIN accomplishes this?

To start things off, the initial setup has been given to you below.  Beneath that, you can fill in your own code.

---

## Connect to a blank database

In [48]:
# Connect to the database and get a cursor to access the data.
conn = connect_db(r':memory:')             # an in-memory database
cur = get_cursor(conn)  # not needed for these examples, but kept for compatibility

Database file does not exist and will be created.
Database connection established.
Cursor created.


## Create four tables by reading data from the Excel file
- Review the Excel file iself to see the source and metadata for the datasets.
- Both .xls and .xlsx versions of the Excel file are provided in the data directory.  Some Python installations may not recognize the .xlsx version, so use what works for you.
- Create four tables: **govt_table**, **internet_table**, **income_eq_table**, **income_perc_table**. The govt_table is already done for you.

In [49]:
create_table_from_excel(conn, '/MiscWikipediaCountryData.xls', 'GovernmentData', 'govt_table')
print_schema(conn,"govt_table")
print_table_or_view(conn, "govt_table", 3) # Show a few example records


SCHEMA: govt_table
[('CREATE TABLE "govt_table" (
"Name" TEXT,
  "UN_Member" INTEGER,
  "Constitutional_Form" TEXT,
  "Head_Of_State" TEXT,
  "Basis_Of_Executive_Legitimacy" TEXT
)',)]

TABLE/VIEW: govt_table
------------------------------------------


,Name,UN_Member,Constitutional_Form,Head_Of_State,Basis_Of_Executive_Legitimacy
0,Abkhazia,0,Republic,Executive,Presidency is independent of legislature
1,Afghanistan,1,Provisional,None,No constitutionally-defined basis to current regime[note 1]
2,Albania,1,Republic,Ceremonial,Ministry is subject to parliamentary confidence


------------------------------------------


In [50]:
create_table_from_excel(conn, '/MiscWikipediaCountryData.xls', 'InternetData', 'internet_table')
print_schema(conn,"internet_table")
print_table_or_view(conn, "internet_table", 3) # Show a few example records


SCHEMA: internet_table
[('CREATE TABLE "internet_table" (
"Country_Or_Area" TEXT,
  "Subregion" TEXT,
  "Region" TEXT,
  "Internet_Users" INTEGER,
  "Pct" REAL,
  "Population" INTEGER,
  "Year" INTEGER
)',)]

TABLE/VIEW: internet_table
------------------------------------------


,Country_Or_Area,Subregion,Region,Internet_Users,Pct,Population,Year
0,Afghanistan,Southern Asia,Asia,4068194,0.101,40099462,2021
1,Albania,Southern Europe,Europe,2105339,0.737,2854710,2021
2,Algeria,Northern Africa,Africa,26350000,0.596,44177969,2021


------------------------------------------


In [51]:
create_table_from_excel(conn, '/MiscWikipediaCountryData.xls', 'IncomeEqualityData', 'income_eq_table')
print_schema(conn,"income_eq_table")
print_table_or_view(conn, "income_eq_table", 3) # Show a few example records


SCHEMA: income_eq_table
[('CREATE TABLE "income_eq_table" (
"Country" TEXT,
  "Subregion" TEXT,
  "Region" TEXT,
  "UN_RP10" REAL,
  "UN_RP20" REAL,
  "CIA_RP10" REAL,
  "CIA_Year" REAL,
  "Gini_Percent" REAL,
  "Gini_Year" REAL
)',)]

TABLE/VIEW: income_eq_table
------------------------------------------


,Country,Subregion,Region,UN_RP10,UN_RP20,CIA_RP10,CIA_Year,Gini_Percent,Gini_Year
0,Afghanistan,Southern Asia,Asia,NaN,NaN,NaN,NaN,NaN,NaN
1,Albania,Southern Europe,Europe,7.2,4.2,7.2,2004.0,30.8,2019.0
2,Algeria,Northern Africa,Africa,9.6,4.0,9.6,1995.0,27.6,2011.0


------------------------------------------


In [52]:
create_table_from_excel(conn, '/MiscWikipediaCountryData.xls', 'IncomePerCapitaData', 'income_perc_table')
print_schema(conn,"income_perc_table")
print_table_or_view(conn, "income_perc_table", 3) # Show a few example records


SCHEMA: income_perc_table
[('CREATE TABLE "income_perc_table" (
"Country_Or_Territory" TEXT,
  "UN_Region" TEXT,
  "IMF_Estimate" REAL,
  "IMF_Year" REAL,
  "WB_Estimate" REAL,
  "WB_Year" REAL,
  "UN_Estimate" REAL,
  "UN_Year" REAL
)',)]

TABLE/VIEW: income_perc_table
------------------------------------------


,Country_Or_Territory,UN_Region,IMF_Estimate,IMF_Year,WB_Estimate,WB_Year,UN_Estimate,UN_Year
0,Afghanistan,Asia,611.0,2020.0,369.0,2021.0,508.0,2020.0
1,Albania,Europe,6369.0,2022.0,6493.0,2021.0,5181.0,2020.0
2,Algeria,Africa,4151.0,2022.0,3691.0,2021.0,3368.0,2020.0


------------------------------------------


---
## Your Code
In the area below, write the SQL code to answer the following questions.  You may add / modify / delete cells as you wish.  Make sure you include comments that describe:
- What you are trying to do, and why.
- How you are doing it (comments in code, if appropriate).
- Your results or findings, if any.

**Answer These Questions:**
1. What is the Population of each Region?
2. Which Constitutional_Form has the highest Population, for counties that have reported their Population?
3. Which Constitutional_Form has the highest average Per_Capita_Income, for counties that have reported their Per_Capita_Income?
4. Which Country has the lowest average Rich_Poor_Income_Ratio, for counties that have reported their Rich_Poor_Income_Ratio?
5. What is the [Region, Subregion, Country, Population] of each of the 10 most populous countries in our dataset?  What is unrealistic about this answer?
6. What are the top 10 Countries in terms of total income (consider Population * Per_Capita_Income)?
---

To start, I want to combine the four separate tables into a single table "common_country_table" with the specific columns of interest

These include: Country, Constitutional_Form, Head_Of_State, Subregion, Region, Population, Rich_Poor_Income_Ratio (using the CIA_RP10 column), Per_Capita_Income (using the IMF_Estimate column)

There is likely a more succinct way to accomplish this, but I will follow these steps:

    1) Combine table 1 (govt_table) and table 2 (internet_table)
        - as "govt_internet_table"
    2) Combine table 3 (income_eq_table) and table 4 (income_perc_table)
        - as "income_eq_perc_table"
    3) Combine govt_internet_table and income_eq_perc_table to create the combined "common_country_table"

In [57]:
run_query(conn,'DROP TABLE IF EXISTS govt_internet_table') # Delete the table if it already exists

# Selecting only the desired columns from govt_table and internet_table
# Using INNER JOIN to remove errors in name values and reduce null fields
result = run_query(conn,'''
    CREATE TABLE IF NOT EXISTS govt_internet_table
    AS SELECT govt_table.name AS Country, govt_table.constitutional_form, govt_table.head_of_state,
        internet_table.subregion, internet_table.region, internet_table.population
    FROM govt_table INNER JOIN internet_table
    ON govt_table.name = internet_table.country_or_area
    WHERE govt_table.un_member = 1
    ''')    # Specifying only records that are UN members

print_table_or_view(conn, "govt_internet_table", 10) # Show 10 records


TABLE/VIEW: govt_internet_table
------------------------------------------


,Country,Constitutional_Form,Head_Of_State,Subregion,Region,Population
0,Afghanistan,Provisional,None,Southern Asia,Asia,40099462
1,Albania,Republic,Ceremonial,Southern Europe,Europe,2854710
2,Algeria,Republic,Executive,Northern Africa,Africa,44177969
3,Andorra,Constitutional monarchy,Ceremonial,Southern Europe,Europe,79034
4,Angola,Republic,Executive,Middle Africa,Africa,34503774
5,Antigua and Barbuda,Constitutional monarchy,Ceremonial,Caribbean,Americas,93219
6,Argentina,Republic,Executive,South America,Americas,45276780
7,Armenia,Republic,Ceremonial,Western Asia,Asia,2790974
8,Australia,Constitutional monarchy,Ceremonial,"Australia, New Zealand",Oceania,25921089
9,Austria,Republic,Ceremonial,Western Europe,Europe,8922082


------------------------------------------


In [58]:
run_query(conn,'DROP TABLE IF EXISTS income_eq_perc_table') # Delete the table if it already exists

# Selecting only the desired columns from income_eq_table and income_perc_table
# Using INNER JOIN to remove errors in name values and reduce null fields
result = run_query(conn,'''
    CREATE TABLE IF NOT EXISTS income_eq_perc_table
    AS SELECT income_eq_table.country, income_eq_table.cia_rp10, income_perc_table.imf_estimate
    FROM income_eq_table INNER JOIN income_perc_table
    ON income_eq_table.country = income_perc_table.country_or_territory
    ''')

print_table_or_view(conn, "income_eq_perc_table", 10) # Show 10 records


TABLE/VIEW: income_eq_perc_table
------------------------------------------


,Country,CIA_RP10,IMF_Estimate
0,Afghanistan,NaN,611.0
1,Albania,7.2,6369.0
2,Algeria,9.6,4151.0
3,Angola,NaN,3791.0
4,Argentina,35.0,13622.0
5,Armenia,25.8,5972.0
6,Australia,12.7,66408.0
7,Austria,6.8,52062.0
8,Azerbaijan,9.5,6842.0
9,Bangladesh,7.5,2734.0


------------------------------------------


In [59]:
run_query(conn,'DROP TABLE IF EXISTS common_country_table') # Delete the table if it already exists

# Renaming appropriate columns to match desired output
# Using INNER JOIN to remove errors in name values and reduce null fields
result = run_query(conn,'''
    CREATE TABLE IF NOT EXISTS common_country_table
    AS SELECT govt_internet_table.*, income_eq_perc_table.cia_rp10 AS Rich_Poor_Income_Ratio, income_eq_perc_table.imf_estimate AS Per_Capita_Income
    FROM govt_internet_table INNER JOIN income_eq_perc_table
    ON govt_internet_table.country = income_eq_perc_table.country
    ''')

print_table_or_view(conn, "common_country_table", 10) # Show 10 records


TABLE/VIEW: common_country_table
------------------------------------------


,Country,Constitutional_Form,Head_Of_State,Subregion,Region,Population,Rich_Poor_Income_Ratio,Per_Capita_Income
0,Afghanistan,Provisional,None,Southern Asia,Asia,40099462,NaN,611.0
1,Albania,Republic,Ceremonial,Southern Europe,Europe,2854710,7.2,6369.0
2,Algeria,Republic,Executive,Northern Africa,Africa,44177969,9.6,4151.0
3,Angola,Republic,Executive,Middle Africa,Africa,34503774,NaN,3791.0
4,Argentina,Republic,Executive,South America,Americas,45276780,35.0,13622.0
5,Armenia,Republic,Ceremonial,Western Asia,Asia,2790974,25.8,5972.0
6,Australia,Constitutional monarchy,Ceremonial,"Australia, New Zealand",Oceania,25921089,12.7,66408.0
7,Austria,Republic,Ceremonial,Western Europe,Europe,8922082,6.8,52062.0
8,Azerbaijan,Republic,Executive,Western Asia,Asia,10312992,9.5,6842.0
9,Bangladesh,Republic,Ceremonial,Southern Asia,Asia,166303498,7.5,2734.0


------------------------------------------


### Now, to answer question 1.

    1. What is the Population of each Region?

In [63]:
# Creating new view if not already generated
# Grouping by region and finding sum of population within that group for total population
result = run_query(conn,'''
    CREATE VIEW IF NOT EXISTS region_pop_view
    AS SELECT region, sum(population) AS Region_Pop
    FROM common_country_table
    GROUP BY region
    ORDER BY region ASC
    ''')

print_table_or_view(conn, "region_pop_view", 5) # Showing a few records


TABLE/VIEW: region_pop_view
------------------------------------------


,Region,region_pop
0,Africa,1247174142
1,Americas,1016133223
2,Asia,3089383089
3,Europe,827755646
4,Oceania,42459469


------------------------------------------


With the outcome table, we can see that the population of each Region is as follows:

Africa: 1,247,174,142

Americas: 1,016,133,223

Asia: 3,089,383,089

Europe: 827,755,646

Oceania: 42,459,469

### Now, for question 2

    2) Which Constitutional_Form has the highest Population, for countries that have reported their Population?

In [65]:
# Creating new view if not already generated
# Grouping by constitutional_form and finding sum of population within that group for total population
result = run_query(conn,'''
    CREATE VIEW IF NOT EXISTS con_form_pop_view
    AS SELECT constitutional_form, sum(population) AS Con_Form_Pop
    FROM common_country_table
    GROUP BY constitutional_form
    ORDER BY con_form_pop DESC
    ''')

print_table_or_view(conn, "con_form_pop_view", 5) # Showing a few records


TABLE/VIEW: con_form_pop_view
------------------------------------------


,Constitutional_Form,Con_Form_Pop
0,Republic,5379988079
1,Provisional,247253701
2,Constitutional monarchy,558521122
3,Absolute monarchy,37142667


------------------------------------------


Based on the output, we can see that the Constitutional Form of **Republic** has the highest population with a total of **5,379,988,079**

### Next, question 3

    3. Which Constitutional_Form has the highest average Per_Capita_Income, for countries that have reported their Per_Capita_Income?

In [66]:
# Creating new view if not already generated
# Grouping by constitutional_form and finding avg of per_capita_income within that group
result = run_query(conn,'''
    CREATE VIEW IF NOT EXISTS con_form_inc_view
    AS SELECT constitutional_form, avg(per_capita_income) AS Avg_Per_Capita_Income
    FROM common_country_table
    GROUP BY constitutional_form
    ORDER BY avg_per_capita_income DESC
    ''')

print_table_or_view(conn, "con_form_inc_view", 5) # Show a few records


TABLE/VIEW: con_form_inc_view
------------------------------------------


,Constitutional_Form,Avg_Per_Capita_Income
0,Constitutional monarchy,36932.000000
1,Absolute monarchy,15998.500000
2,Republic,13353.570248
3,Provisional,909.750000


------------------------------------------


The Constitutional Form with the highest average Per Capita Income is shown to be **Constitutional Monarchy** with an average per capita income of **$36,932**

### Now, question 4

    4. Which Country has the lowest average Rich_Poor_Income_Ratio, for countries that have reported their Rich_Poor_Income_Ratio?

In [68]:
# Creating new view if not already generated
# Selecting country and rich_poor_income_ratio and putting in ascending order to show the lowest value
result = run_query(conn,'''
    CREATE VIEW IF NOT EXISTS avg_rpi_view
    AS SELECT country, rich_poor_income_ratio
    FROM common_country_table
    WHERE rich_poor_income_ratio <> "NaN"
    ORDER BY rich_poor_income_ratio ASC
    ''')    # Using WHERE <> "NaN" to remove any countries that did not provide a value

print_table_or_view(conn, "avg_rpi_view", 5) # Show a few records


TABLE/VIEW: avg_rpi_view
------------------------------------------


,Country,Rich_Poor_Income_Ratio
0,Japan,4.5
1,Czech Republic,5.2
2,Bosnia and Herzegovina,5.5
3,Hungary,5.6
4,Finland,5.7


------------------------------------------


This output shows that **Japan** is the country with the lowest average Rich Poor Income Ratio with a ratio of **4.5**

### Now, question 5

    5. What is the [Region, Subregion, Country, Population] of each of the 10 most populous countries in our dataset?
        What is unrealistic about the answer?

In [69]:
# Creating new view if not already generated
# Selecting columns of interest and ordering by population in descending order
result = run_query(conn,'''
    CREATE VIEW IF NOT EXISTS pop_info_view
    AS SELECT region, subregion, country, population
    FROM common_country_table
    WHERE population <> "NaN"
    ORDER BY population DESC
    ''')    # Using WHERE <> "NaN" to remove any countries that did not provide a value

print_table_or_view(conn, "pop_info_view", 10) # Show 10 records as specified


TABLE/VIEW: pop_info_view
------------------------------------------


,Region,Subregion,Country,Population
0,Asia,Southern Asia,India,1407563842
1,Americas,Northern America,United States,336997624
2,Asia,South-eastern Asia,Indonesia,273753191
3,Americas,South America,Brazil,214326223
4,Asia,Southern Asia,Pakistan,213756286
5,Africa,Western Africa,Nigeria,213401323
6,Asia,Southern Asia,Bangladesh,166303498
7,Europe,Eastern Europe,Russia,145102755
8,Americas,Northern America,Mexico,128972439
9,Asia,Eastern Asia,Japan,124612530


------------------------------------------


Listed above are the 10 most populous countries from this dataset. It would be realistic to simply conclude that these are the 10 most populous from **this dataset** in particular. However, it would be unrealistic to try and generalize this data outside of this context due to multiple possible factors.

1) Countries may have not provided population data (notably China, which should be very high on the list)
2) Countries may have been misspelled or formatted incorrectly, resulting in them not being included in the combined table used for the analysis
3) This exclused any country outside of the UN, so we cannot assume that this reflects anything other than most populous countries in the UN.

### Lastly, question 6

    6. What are the top 10 Countries in terms of total income (consider Population * Per_Capita_Income)?

In [71]:
# Creating new view if not already generated
# Selecting columns of interest and creating new column for income
result = run_query(conn,'''
    CREATE VIEW IF NOT EXISTS income_info_view
    AS SELECT country, (population * per_capita_income) AS Total_Income
    FROM common_country_table
    WHERE population <> "NaN" AND per_capita_income <> "NaN"
    ORDER BY total_income DESC
    ''')    # Using WHERE <> "NaN" to remove any countries that did not provide a value

print_table_or_view(conn, "income_info_view", 10) # Show 10 records as specified


TABLE/VIEW: income_info_view
------------------------------------------


,Country,Total_Income
0,United States,2.533548e+13
1,Japan,4.281437e+12
2,Germany,4.036807e+12
3,India,3.471052e+12
4,United Kingdom,3.183604e+12
5,France,2.731616e+12
6,Canada,2.166976e+12
7,Russia,2.127932e+12
8,Iran,2.025228e+12
9,Italy,1.998769e+12


------------------------------------------


---
# Cleanup and Close the Current Database
---

In [72]:
close_db(conn, cur)

***
# Section: Conclusions
- What did you learn from this exercise?
***

I feel that this was a very effective way of practicing manipulating the SQL tables and finding specific outcomes and solutions to questions at hand. This definitely helped in understanding the differences in JOIN options as I compared this assignment to the example notebook and the different ways in which they were used. I kept thinking that there were possibly quicker ways to accomplish things, but I tried to limit those thoughts and just approach it in an incremental way that I could easily make sense of and reach the conclusion as efficiency is not really required in this situation. The syntax was easy to follow (pending my answers aren't all incorrect) so I was able to work through these questions without much second-guessing or looking to my resources. All in all, it was a good exercise in SQL basics!